In [14]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [15]:
import numpy as np
import pandas as pd

## 1. groupby의 기본 예제

In [16]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                    "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                        dtype="Int64"),
                    "data1" : np.random.standard_normal(7),
                    "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [17]:
df["data1"]

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
5    1.393406
6    0.092908
Name: data1, dtype: float64

In [18]:
df["key1"]

0       a
1       a
2    None
3       b
4       b
5       a
6    None
Name: key1, dtype: object

In [19]:
grouped = df["data1"].groupby(df["key1"])   # groupby에서 None값은 무시됨
grouped

In [20]:
grouped.mean()

key1
a    0.555881
b    0.705025
Name: data1, dtype: float64

In [21]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.204708
      2       0.478943
b     1       1.965781
      2      -0.555730
Name: data1, dtype: float64

### 피봇테이블 예시

In [22]:
means.unstack()   # 가장 높은 레벨의 인덱스를 열로 옮김

key2,1,2
key1,,
a,-0.204708,0.478943
b,1.965781,-0.555730


### 다른 예시

In [23]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])   # 넘파이 배열
years  = [2005, 2005, 2006, 2005, 2006, 2005, 2006]             # 리스트
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [24]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.936175
    2006   -0.519439
OH  2005   -0.380219
    2006    1.029344
Name: data1, dtype: float64

#### 데이터 프레임에 대해 groupby 적용 (위는 series에 적용)

In [25]:
df.groupby("key1").mean()   # 키값으로 groupby 가능

,key2,data1,data2
key1,,,
a,1.5,0.555881,0.441920
b,1.5,0.705025,-0.144516


In [26]:
df.groupby("key2").mean(numeric_only=True)  # 숫자로 이루어진 피쳐만 나타냄

,data1,data2
key2,,
1,0.333636,0.115218
2,-0.038393,0.888106


In [27]:
df.groupby("key2").mean()       # 버전에 따라 오류일수도 아닐수도..

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

### groupby 객체의 size() 메소드

In [28]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [29]:
df.groupby(["key1", "key2"]).size()     # 몇 개 있는지 알려줌

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [30]:
df.groupby("key1", dropna=False).size()     # 결측치 포함

key1
a      3
b      2
NaN    2
dtype: int64

In [31]:
df.groupby(["key1", "key2"], dropna=False).size()   # 결측치 포함

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

### groupby객체의 count() 메소드

In [32]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [33]:
df.groupby("key1").count()  # 결측값 제외하고 'key1'이 각각의 column에 몇개 있는지

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## 2. 그룹간 순회하기

### 그룹 이름과 해당 그룹으로 구성된 데이터프레임을 반환

In [34]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [35]:
# 그룹 이름, 해당 그룹으로 구성된 데이터 프레임
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [36]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
('a', 2)
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
('b', 1)
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
('b', 2)
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


### 딕셔너리를 만들어서 활용 가능

In [37]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221


## 3. 딕셔너리를 통해 그룹화 하는 방법

In [38]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [39]:
grouped = df.groupby({0:"low", 1:"low",
                        2:"low", 3:"low",4:"high", 5:"high",
                        6:"high"})
# grouped = df.groupby(["low", "low", "low", "low","high", "high", "high"])     # 이 방법으로도 가능
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

high
   key1  key2     data1     data2
4     b     1  1.965781 -1.296221
5     a  <NA>  1.393406  0.274992
6  None     1  0.092908  0.228913
low
   key1  key2     data1     data2
0     a     1 -0.204708  0.281746
1     a     2  0.478943  0.769023
2  None     1 -0.519439  1.246435
3     b     2 -0.555730  1.007189


### 열을 그룹화 하는 방법 

In [40]:
grouped = df.groupby({"key1": "key", "key2": "key",
                        "data1": "data", "data2": "data"}, axis="columns")

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/3213942183.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [41]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.204708  0.281746
1  0.478943  0.769023
2 -0.519439  1.246435
3 -0.555730  1.007189
4  1.965781 -1.296221
5  1.393406  0.274992
6  0.092908  0.228913
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


## 4. 열의 일부 선택하기

In [42]:
df.groupby(["key1", "key2"])[["data2"]].mean()  #data1은 선택되지 않음

data2
key1 key2          
a    1     0.281746
     2     0.769023
b    1    -1.296221
     2     1.007189

In [43]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [44]:
s_grouped.mean()

key1  key2
a     1       0.281746
      2       0.769023
b     1      -1.296221
      2       1.007189
Name: data2, dtype: float64

## 5. 다양한 방식을 그룹화하기

In [45]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


### 딕셔너리를 통한 그룹화

In [46]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [47]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-2.373480,3.908371
Steve,3.725929,-1.999539
Wanda,0.523772,-0.576147
Jill,-3.201385,-1.230495
Trey,-1.146107,-1.364125


### 시리즈를 통한 그룹화

In [48]:
map_series = pd.Series(mapping)
print(map_series)
people.groupby(map_series, axis="columns").count()

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object


/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/1010316084.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### 함수를 통한 그룹화 - 함수는 index에 적용됨

In [49]:
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


In [50]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,0.483052,-0.153399,-2.097088,-2.250405,-2.924273
5,-1.015657,-0.539741,0.476985,3.772716,-1.020287


### 함수와 시리즈를 섞어서 그룹화

In [51]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,two,-0.860757,-0.713544,-1.265934,-2.370232,-1.860761
5,one,-0.577087,-0.539741,0.476985,0.523772,-1.021228


## 6. 데이터 집계

![](images/groupby메소드.jpg)

In [52]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [53]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)   # 가장 작은 것부터 정렬했을 때 2개 반환

key1   
a     0   -0.204708
      1    0.478943
b     3   -0.555730
      4    1.965781
Name: data1, dtype: float64

In [54]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

# key1=a일 때, key2의 max-min=2-1=1, 
# data1의 max-min=1.39xx-(-0.20xx)=1.59xx
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


## 7. 데이터 집계 관련 다양한 예시

In [88]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [89]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [90]:
grouped = tips.groupby(["day", "smoker"])

### agg(메소드 이름 혹은 함수) 를 통해서도 집계 가능

In [91]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean") 
# 자체로 만든 함수가 아닌 groupby객체의 메소드이므로 ""라고 표현

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

### 메소드 이름과 함수 배열을 통해 여러 집계 값을 나타낼 수 있음

In [92]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

### 튜플 형태로 적용되는 함수의 이름을 나타낼 수 있음

In [93]:
# group.agg([("이름1", "함수"), ("이름2", 객체메소드)])
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/42984094.py:2: FutureWarning: The provided callable <function std at 0x105853920> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

### 여러 개의 열에 적용 가능 하고,  딕셔너리를 통해서 열마다 서로 다른 함수도 적용 가능

In [94]:
for name, df in grouped:
    print(name)
    print(df)

('Fri', 'No')
     total_bill   tip smoker  day    time  size   tip_pct
91        22.49  3.50     No  Fri  Dinner     2  0.155625
94        22.75  3.25     No  Fri  Dinner     2  0.142857
99        12.46  1.50     No  Fri  Dinner     2  0.120385
223       15.98  3.00     No  Fri   Lunch     3  0.187735
('Fri', 'Yes')
     total_bill   tip smoker  day    time  size   tip_pct
90        28.97  3.00    Yes  Fri  Dinner     2  0.103555
92         5.75  1.00    Yes  Fri  Dinner     2  0.173913
93        16.32  4.30    Yes  Fri  Dinner     2  0.263480
95        40.17  4.73    Yes  Fri  Dinner     4  0.117750
96        27.28  4.00    Yes  Fri  Dinner     2  0.146628
97        12.03  1.50    Yes  Fri  Dinner     2  0.124688
98        21.01  3.00    Yes  Fri  Dinner     2  0.142789
100       11.35  2.50    Yes  Fri  Dinner     2  0.220264
101       15.38  3.00    Yes  Fri  Dinner     2  0.195059
220       12.16  2.20    Yes  Fri   Lunch     2  0.180921
221       13.42  3.48    Yes  Fri   Lunch  

In [95]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [96]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/4198102554.py:2: FutureWarning: The provided callable <function var at 0x105853a60> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [97]:
# 열마다 다른 함수 적용
grouped.agg({"tip" : np.max, "size" : "sum"})

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/2082172191.py:2: FutureWarning: The provided callable <function max at 0x105852de0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip" : np.max, "size" : "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [98]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
            "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [99]:
grouped = tips.groupby(["day", "smoker"])
grouped.mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [100]:
grouped = tips.groupby(["day", "smoker"], as_index=False)  
## as_index 옵션을 통해서 인덱스를 그룹화된 인덱스로 바꾸지 X
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 8. apply 메서드

groupby의 agg, mean, max 등 집계 방법과 가장 큰 차이  
-> apply는 열 별로 집계하지 않고  
    입력값이 그룹화된 데이터 프레임으로 모든 행, 열을 동시 고려

In [65]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [66]:
# 각 그룹에 top이라는 함수 적용
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [67]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill") 

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### group_keys=False를 통해서 인덱싱 작업을 하지 않을 수 있음

In [68]:
tips.groupby(["smoker", "day"], group_keys=False).apply(top, n=1, column="total_bill")  

,total_bill,tip,smoker,day,time,size,tip_pct
94,22.75,3.25,No,Fri,Dinner,2,0.142857
212,48.33,9.00,No,Sat,Dinner,4,0.186220
156,48.17,5.00,No,Sun,Dinner,6,0.103799
142,41.19,5.00,No,Thur,Lunch,5,0.121389
95,40.17,4.73,Yes,Fri,Dinner,4,0.117750
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
182,45.35,3.50,Yes,Sun,Dinner,3,0.077178
197,43.11,5.00,Yes,Thur,Lunch,4,0.115982


## 9. groupby를 활용한 예시 1 - 사분위수 분석, 버킷 분석

In [70]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-1.338659,-0.119117
1,0.371224,-0.560719
2,1.428753,-0.284725
3,-0.592818,1.761764
4,-0.312449,1.009692


In [71]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0    (-1.908, -0.387]
1     (-0.387, 1.133]
2      (1.133, 2.654]
3    (-1.908, -0.387]
4     (-0.387, 1.133]
5     (-0.387, 1.133]
6     (-0.387, 1.133]
7    (-1.908, -0.387]
8     (-0.387, 1.133]
9     (-0.387, 1.133]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.434, -1.908] < (-1.908, -0.387] < (-0.387, 1.133] < (1.133, 2.654]]

In [72]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)  ## category를 groupby의 기준으로 삼을 수 있음
grouped.apply(get_stats)

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_2588/2836935835.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)  ## category를 groupby의 기준으로 삼을 수 있음


min       max  count      mean
data1                                                      
(-3.434, -1.908] data1 -3.428254 -1.941714     25 -2.310845
                 data2 -2.079446  1.388465     25 -0.082462
(-1.908, -0.387] data1 -1.902298 -0.393984    323 -0.954838
                 data2 -2.909373  2.531127    323 -0.047416
(-0.387, 1.133]  data1 -0.387236  1.129965    516  0.319806
                 data2 -3.548824  3.366626    516  0.034026
(1.133, 2.654]   data1  1.134073  2.653656    136  1.577425
                 data2 -2.091554  2.615416    136  0.052566

In [74]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.434, -1.908] -3.428254 -1.941714    25 -2.310845 -2.079446  1.388465   
(-1.908, -0.387] -1.902298 -0.393984   323 -0.954838 -2.909373  2.531127   
(-0.387, 1.133]  -0.387236  1.129965   516  0.319806 -3.548824  3.366626   
(1.133, 2.654]    1.134073  2.653656   136  1.577425 -2.091554  2.615416   

                                  
                 count      mean  
data1                             
(-3.434, -1.908]    25 -0.082462  
(-1.908, -0.387]   323 -0.047416  
(-0.387, 1.133]    516  0.034026  
(1.133, 2.654]     136  0.052566

In [77]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    0
1    2
2    3
3    1
4    1
Name: data1, dtype: int64

In [78]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.428254 -0.702558    250 -1.252237
      data2 -2.909373  2.531127    250 -0.090336
1     data1 -0.697173  0.014029    250 -0.331095
      data2 -3.548824  2.419003    250  0.032257
2     data1  0.016520  0.722659    250  0.361084
      data2 -2.611124  3.366626    250  0.034712
3     data1  0.722856  2.653656    250  1.275712
      data2 -2.748685  2.615416    250  0.052686

## 10. groupby를 활용한 예시 2 - 그룹별 값으로 결측치 채우기

In [79]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1    0.130148
2         NaN
3   -0.439194
4         NaN
5    1.543588
dtype: float64

In [80]:
s.fillna(s.mean())

0    0.411514
1    0.130148
2    0.411514
3   -0.439194
4    0.411514
5    1.543588
dtype: float64

In [82]:
states = ["Ohio", "New York", "Vermont", "Florida",
        "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
            "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          2.162370
New York     -0.703736
Vermont      -0.703728
Florida       0.811639
Oregon        0.684245
Nevada       -1.338431
California    1.880710
Idaho         1.328141
dtype: float64

In [83]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          2.162370
New York     -0.703736
Vermont            NaN
Florida       0.811639
Oregon        0.684245
Nevada             NaN
California    1.880710
Idaho              NaN
dtype: float64

In [84]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [85]:
data.groupby(group_key).count()     # 결측치 제외하고 count

East    3
West    2
dtype: int64

In [86]:
data.groupby(group_key).mean()

East    0.756758
West    1.282477
dtype: float64

In [87]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio          2.162370
      New York     -0.703736
      Vermont       0.756758
      Florida       0.811639
West  Oregon        0.684245
      Nevada        1.282477
      California    1.880710
      Idaho         1.282477
dtype: float64

In [110]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])                ## 각 group의 name attribute가 있음

data.groupby(group_key).apply(fill_func)

East  Ohio          2.162370
      New York     -0.703736
      Vermont       0.500000
      Florida       0.811639
West  Oregon        0.684245
      Nevada       -1.000000
      California    1.880710
      Idaho        -1.000000
dtype: float64

## 11. groupby를 활용한 예시 3 - 랜덤 표본과 순열

In [113]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [114]:
deck.head(15)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
dtype: int64

In [119]:
def draw(deck, n=5):
    return deck.sample(n)   # 시리즈의 메소드 중 하나 sample
draw(deck)

9H      9
8H      8
2D      2
10S    10
5C      5
dtype: int64

In [123]:
def get_suit(card):
    return card[-1]

# 숫자를 제외한 뒷 글자(suit)로 groupby
deck.groupby(get_suit).apply(draw, n=2)

C  KC     10
   4C      4
D  8D      8
   10D    10
H  7H      7
   2H      2
S  KS     10
   8S      8
dtype: int64

In [124]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C     9
QC    10
3D     3
7D     7
QH    10
9H     9
6S     6
9S     9
dtype: int64

## 12. transform 

In [157]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [158]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [159]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [160]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [161]:
g.apply('mean')

TypeError: 'str' object is not callable

In [162]:
g.apply(get_mean)

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [163]:
def times_two(group):
    return group * 2
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [164]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [165]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [166]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [168]:
g.apply(normalize)  ## 같은 결과를 얻음

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 13. 피벗 테이블

In [196]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [171]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"]) ## 기본은 평균치를 집계함 

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [175]:
tips.pivot_table(index=["time", "day"], columns=["smoker","size"],
                 values=["tip_pct"])

tip_pct                                                    \
smoker             No                                                     
size                1         2         3         4         5         6   
time   day                                                                
Dinner Fri        NaN  0.139622       NaN       NaN       NaN       NaN   
       Sat   0.137931  0.162705  0.154661  0.150096       NaN       NaN   
       Sun        NaN  0.168859  0.152663  0.148143  0.206928  0.103799   
       Thur       NaN  0.159744       NaN       NaN       NaN       NaN   
Lunch  Fri        NaN       NaN  0.187735       NaN       NaN       NaN   
       Thur  0.181728  0.166005  0.084246  0.138919  0.121389  0.173706   

                                                               
smoker            Yes                                          
size                1         2         3         4         5  
time   day                                                     
Dinner Fri        NaN  0.171297       NaN  0.117750       NaN  
       Sat   0.325733  0.148668  0.144995  0.124515  0.106572  
       Sun        NaN  0.207893  0.152660  0.193370  0.065660  
       Thur       NaN       NaN       NaN       NaN       NaN  
Lunch  Fri   0.223776  0.181969       NaN       NaN       NaN  
       Thur       NaN  0.158843  0.204952  0.155410       NaN

In [176]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct","size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [177]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [186]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [187]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc="count", margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [188]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

## 14. 교차표 (crosstab)

In [189]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

In [190]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [191]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [215]:
data.pivot_table(index='Nationality', columns='Handedness',values='Sample',aggfunc='count',margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [192]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [213]:
tips.pivot_table(index=['time','day'], columns='smoker', values='total_bill',aggfunc='count',margins=True)

smoker          No   Yes  All
time   day                   
Dinner Fri     3.0   9.0   12
       Sat    45.0  42.0   87
       Sun    57.0  19.0   76
       Thur    1.0   NaN    1
Lunch  Fri     1.0   6.0    7
       Thur   44.0  17.0   61
All          151.0  93.0  244